In [17]:
from huggingface_hub import hf_hub_download
from datasets import load_dataset, load_from_disk
from pathlib import Path

BASE_DATA_DIR = Path("./data")
BASE_DATA_DIR.mkdir(parents=True, exist_ok=True)

In [18]:
REPO_ID = "BUT-FIT/BUT-LCC"
FILE_NAME = "train_0.jsonl.gz"

dataset_path = hf_hub_download(repo_id=REPO_ID, filename=FILE_NAME, repo_type="dataset")
dataset = load_dataset('json', data_files=dataset_path, split='train')

Generating train split: 17678059 examples [01:05, 268624.23 examples/s]


In [19]:
dataset = dataset.filter(lambda x: x["part"] == "cswiki-20230101")

Filter: 100%|██████████| 17678059/17678059 [02:04<00:00, 141551.37 examples/s]


In [20]:
dataset.save_to_disk(BASE_DATA_DIR / "cs-wiki")

load_from_disk(BASE_DATA_DIR / "cs-wiki")

Saving the dataset (1/1 shards): 100%|██████████| 50406/50406 [00:10<00:00, 4931.33 examples/s]


Dataset({
    features: ['id', 'part', 'title', 'text', 'ugly', 'ugly_score'],
    num_rows: 50406
})